# 知识工程-作业5 潜在语义索引构建

2024214500 叶璨铭


## 代码与文档格式说明

> 本文档使用Jupyter Notebook编写，所以同时包括了实验文档和实验代码。

> 本次实验项目采用了类似于 Quarto + nbdev 的方法来同步Jupyter Notebook代码到python文件, 因而我们的实验文档导出为pdf和html格式可以进行阅读，而我们的代码也导出为python模块形式，可以作为代码库被其他项目使用。
我们这样做的好处是，避免单独管理一堆 .py 文件，防止代码冗余和同步混乱，py文件和pdf文件都是从.ipynb文件导出的，可以保证实验文档和代码的一致性。

> 本文档理论上支持多个格式，包括ipynb, html, docx, pdf, md 等，但是由于 quarto和nbdev 系统的一些bug，我们目前暂时只支持ipynb文件，以后有空的时候解决bug可以构建一个[在线文档网站](https://thu-coursework-machine-learning-for-big-data-docs.vercel.app/)。您在阅读本文档时，可以选择您喜欢的格式来进行阅读，建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。


> 为了记录我们自己修改了哪些地方，使用git进行版本控制，这样可以清晰地看出我们基于助教的代码在哪些位置进行了修改，有些修改是实现了要求的作业功能，而有些代码是对助教的代码进行了重构和优化。我将我在知识工程课程的代码，在作业截止DDL之后，开源到 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering.git ，方便各位同学一起学习讨论。


## 代码规范说明

为了让代码清晰规范，在作业开始前，使用 `ruff format`格式化助教的代码; 在我们实现函数过程中，函数的docstring应当遵循fastai规范而不是numpy规范，这样简洁清晰，不会Repeat yourself。

```bash
❯ ruff format
error: Failed to parse util.py:5:34: Multiple return types must be parenthesized
2 files reformatted
```

原来，老师给的`util.py:5:34`代码这里类型标注不规范，稍作修改即可
```python
-def load_words(filepath: str) -> List[str], List[List[str]]:
+def load_words(filepath: str) -> Tuple[List[str], List[List[str]]]:
```

此外，原本代码的类型不严谨，语义不规范，导致VSCode报了很多错，我们先重构一下助教的代码，增加合适的注释和类型提示。

清晰的类型注解也是能够帮助我们更好的理解代码的，提高我们对作业的理解，所以不惜花一点时间。


比如 import * 不是很规范。

![alt text](image.png)

我们简单修复一下即可

```python
-from typing import *
+from typing import Tuple, List
```

原本的代码多次错误地使用了 `np.array` 来标注类型，这是不对的，应该使用 `np.ndarray` 来标注类型。比如这个地方

```python
-def cal_tfidf_matrix(term_doc: np.array, documents: List[List[str]], terms: List[str]):
+def cal_tfidf_matrix(term_doc: np.ndarray, documents: List[List[str]], terms: List[str]):
```

此外这个地方应该用多了, 实际上我们这次作业不需要这个标准库，而且python3.12不建议用。
```python
-import imp
```

```bash
/home/ye_canming/repos/assignments/THU-Coursework-Knowledge-Engineering/5.潜在语义索引构建的作业/main.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
```


这下我们修改完，pylance不再报错了，可以安心写主要逻辑了。


## 原理回顾和课件复习



### 向量空间模型

共现矩阵（co-occurrence matrix）有两种，我们这次作业用的是 Term-Document 矩阵，也就是行代表每一个单词，列代表每一个文档，值表示出现次数，文档被这里的列向量表达用来检索，反过来看，每个单词的语义也被行向量所表达。

而Term-Term则是上下文同时出现(小窗口，左右4个单词)两个词的次数，从而获得词向量，可以表示单词的相似性。


### 潜在语义分析

用特征来代替词，从数据中学习特征。本来是为了解决同义词不好检索的问题,一义多词(synonymy)。

Term Document Matrix A 有 m个单词，n个文档，单词多过文档，rank r

$A = U \sum V^T $, 

SVD可以截断k，是F范数下的最优近似。

#### 潜在语义索引（LSI）

就是直接把 U mxr 当做 m个单词的行向量，V rxn 当做 文档的列向量。
这就是把文档和单词都变到了 R维的 隐性空间 （主题空间）

然后用余弦值作为相似度

如果词频矩阵稀疏，SVD计算很难，效果不好。

#### 缺点

不能解决一词多义(polysemy)，因为单词只有一个向量。

SVD是最优 L2 范数近似，但是非负向量的分布不像是高斯分布。

忽略词语先后顺序。

文档和单词的出现应该是泊松分布？




#### 概率LSA
文档概率分布 P(d)
特定文档的话题分布 P(z|d)
特定话题的的单词分布 P(w|z)
特定文档的单词分布 P(w|d)

![alt text](image-1.png)

一个共现矩阵T出现的概率，就是 单词-文档 对出现概率乘上出现次数。




## 数据加载与结巴分词



我们看下 main.py 文件

使用到数据的地方是

```python
terms_tmp, documents_tmp = load_words(os.path.join(filePath, file))

```


首先我们确诊一下数据的格式

![alt text](image-2.png)
从VSCode的警告来看，这个文件未必就是UTF-8，所以filePath 的文本编码不确定，所以我们来个 pip install chardet



现在我们来实现这个函数。

In [ ]:
import joblib

memory = joblib.Memory(location="cache", verbose=0)


@memory.cache
def load_words(
    filepath: str,  # 要加载的文件路径
) -> Tuple[List[str], List[List[str]]]:  # (所有词汇列表, 文档分词列表)
    """读取文件并使用jieba分词"""
    with open(filepath, "rb") as f:
        raw_data = f.read()
        # 使用 chardet 检测编码
        result = chardet.detect(raw_data)
    encoding = result["encoding"]
    confidence = result["confidence"]
    if confidence < 0.7:
        print(
            f"Warning: low confidence ({confidence}) for encoding{encoding}. Using utf-8 instead."
        )
        encoding = "utf-8"
    print(
        f"Loading file {filepath} with encoding {encoding}, confidence {confidence}. "
    )

    with open(filepath, "r", encoding=encoding) as f:
        documents = []
        all_terms = set()

        for line in f:
            line = line.strip()
            if not line:
                continue

            # 使用jieba分词
            words = jieba.lcut(line)
            documents.append(words)
            all_terms.update(words)

    return all_terms, documents

其中注意jieba用得是 lcut 接口，而不是我们作业1用的tokenize接口。

需要注意的是由于外面已经排过序，不需要在这里对terms排序, 只需要正确返回列表即可。

为了避免运行太多次，我们使用joblib进行了 cache

**我们这里认为一个句子是一个document！**

## 矩阵构建与TF-IDF

首先实现 term_doc = build_term_doc_matrix(documents, terms)·

这个没什么特别的，主要是要把索引做对，之所以对term索引而不是对doc索引，是因为 doc是 List[List[str]] 要被遍历。


**特别注意这里和理论课课件的方向有所不同**
每一行是文档，每一列是单词

In [ ]:
def build_term_doc_matrix(
    documents: List[List[str]],  # 文档分词列表 (N个文档)
    terms: List[str],  # 所有词汇列表 (D个词汇)
) -> np.ndarray:  # N×D的矩阵，term_doc[i,j]表示词j在文档i中的出现次数
    """
    build term-document matrix
    """
    # 创建词汇到索引的映射
    term_index = {term: idx for idx, term in enumerate(terms)}

    # 初始化矩阵
    matrix = np.zeros((len(documents), len(terms)), dtype=int)

    # 填充矩阵
    for doc_idx, doc in enumerate(documents):
        for word in doc:
            if word in term_index:
                matrix[doc_idx, term_index[word]] += 1

    return matrix

现在写 cal_tfidf_matrix

```python
TF_IDF = cal_tfidf_matrix(term_doc, documents, terms)
    print(
        "TOP10单词的TF-TDF如下所示\n",
        sorted(TF_IDF.items(), key=lambda x: x[1], reverse=True)[:10],
    )
```

课件上没有强调这个，所以我们补充阅读一下 https://zh.wikipedia.org/zh-hans/Tf-idf

所谓 TF，term frequency，就是归一化，对document这个维度，去对term做归一化，让每个document的向量都是L1范数为1的向量。
所谓IDF，就是 inverse document frequency，是想说，一个词如果所有文档都有就不重要了。所以就是横向地看在所有文档中，这个单词出现（>0就行）的文档数量，因为要作为重要性，所以1/这个数量。不过至于为什么还有一个 log |D| 我不太明白。

所谓 TFIDF，就是TF*IDF。

注意如果有单词没有出现，但是在词表里面，那IDF有可能出现定义失败的情况，所以增加参数 epsilon


In [ ]:
def cal_tfidf_matrix(
    term_doc: np.ndarray,  # 词-文档矩阵
    documents: List[List[str]],
    terms: List[str],
    epsilon: float = 1e-6,  # 增加一个很小的数，避免除以0
) -> dict:  # TF-IDF值字典 {word: tfidf_value}
    """
    calculate TF-IDF value for each word
    """
    # TF计算
    tf = term_doc.astype(float)  # 原本是int
    doc_lens = np.sum(tf, axis=1)  # 一共多少个词，在axis1上面求和，消灭axis1
    tf = tf / doc_lens  # 归一化

    # IDF计算
    df = (term_doc > 0).sum(axis=0)  # 每一行是文档，把文档消除，看看单词有出现的次数。
    idf = np.log(len(documents) / (df + epsilon))  # 避免除以0

    # TF-IDF计算
    tfidf = tf * idf

    # 转换为字典
    return {term: tfidf[:, idx].mean() for idx, term in enumerate(terms)}

## 相似度计算

接下来实现 search_key_similarity


注意助教代码中，keys = []是从文件名得到的查询关键词，这个没有经过结巴分词，首先我们就需要分词。


In [ ]:
def search_key_similarity(
    U: np.ndarray,  # SVD分解的U矩阵
    s: np.ndarray,  # 奇异值数组
    VT: np.ndarray,  # SVD分解的VT矩阵
    terms: List[str],  # 词汇列表
    term_doc: np.ndarray,
    keys: List[str],  # 查询关键词列表
    k: int = 10,  # 保留的奇异值数量
) -> np.ndarray:  # 相似度矩阵 (N文档, K关键词)
    """
    计算LSI相似度矩阵
    """
    # 构建查询向量
    term_index = {term: idx for idx, term in enumerate(terms)}
    query_vectors = []

    for key in keys:
        words = jieba.lcut(key)
        vec = np.zeros(len(terms))
        for word in words:
            if word in term_index:
                vec[term_index[word]] += 1
        query_vectors.append(vec)

    query_matrix = np.array(query_vectors).T  # D x K

    # 降维处理
    sigma_k = np.diag(s[:k])
    U_k = U[:, :k]
    VT_k = VT[:k, :]

    # 文档在隐空间中的表示
    doc_rep = U_k @ sigma_k  # N x k

    # 查询在隐空间中的表示
    # query_rep = np.linalg.pinv(sigma_k) @ VT_k @ query_matrix  # k x K
    sigma_k_inv = np.diag(1.0 / np.diag(sigma_k))
    query_rep = sigma_k_inv @ VT_k @ query_matrix

    # 计算余弦相似度
    return cosine_similarity(doc_rep, query_rep.T)  # 直接用sklearn的余弦相似度计算

注意表达式上是-1，但是对角矩阵和正交矩阵不用那么复杂。


分类自然也很简单

In [ ]:
def classification(
    sim_matrix: np.ndarray,  # 相似度矩阵 (N文档, K关键词)
) -> np.ndarray:  # 预测的类别索引 (N文档,)
    """
    文档分类：为每个文档选择最相似的关键词
    """
    return np.argmax(sim_matrix, axis=1)

In [ ]:
def search_topn_for_each_key(
    sim_matrix: np.ndarray,  # 相似度矩阵 (N文档, K关键词)
    n: int = 10,  # 保留的Top-N结果数量
) -> np.ndarray:  # 搜索结果矩阵 (K关键词, n文档索引)
    """
    为每个关键词搜索Top-N文档
    """
    # 按列排序（每个关键词对应一列）
    return np.argsort(-sim_matrix, axis=0)[:n, :].T

## 运行结果

In [5]:
!python main.py

Loading file ./data/意大利封闭全国.txt with encoding utf-8, confidence 0.99. 
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.878 seconds.
Prefix dict has been built successfully.
Loading file ./data/戈贝尔确诊新冠.txt with encoding utf-8, confidence 0.99. 
Loading file ./data/欧洲杯推迟.txt with encoding utf-8, confidence 0.99. 
Loading file ./data/汤姆汉克斯确诊.txt with encoding utf-8, confidence 0.99. 
Loading file ./data/特鲁多自我隔离.txt with encoding utf-8, confidence 0.99. 
Loading file ./data/疫情防控思政大课.txt with encoding utf-8, confidence 0.99. 
Loading file ./data/美国从阿富汗撤军.txt with encoding utf-8, confidence 0.99. 
Loading file ./data/英国央行紧急降息.txt with encoding utf-8, confidence 0.99. 
Loading file ./data/苹果折叠手机专利.txt with encoding utf-8, confidence 0.99. 
Loading file ./data/魔兽世界怀旧服.txt with encoding utf-8, confidence 0.99. 
TOP10单词的TF-TDF如下所示
 [('阿富汗', 0.006327252874692564), ('欧洲杯', 0.005215631936116387), ('折叠', 0.0050477958652063015), ('意大

为了修改参数n查看效果，我们增加argparse

In [ ]:
import argparse
parser = argpawrse.ArgumentParser()
parser.add_argument("--n", type=int, default=5)
args = parser.parse_args()

我们用 n=10 试一下

In [1]:
!python main.py --n 10


TOP10单词的TF-TDF如下所示
 [('阿富汗', 0.006327252874692564), ('欧洲杯', 0.005215631936116387), ('折叠', 0.0050477958652063015), ('意大利', 0.0050270258549414346), ('央行', 0.004862996707500968), ('任务', 0.004792787625878285), ('英国', 0.004705536630277911), ('戈贝尔', 0.004662253844871659), ('特鲁多', 0.004567633613773077), ('塔利班', 0.004254171083783253)]
U: (652, 652)
s: (652,)
VT: (23782, 23782)
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.581 seconds.
Prefix dict has been built successfully.
[[0 5 3 2 0 6 4 0 2 0]
 [4 3 1 3 1 4 1 0 1 1]
 [1 2 1 1 1 1 2 1 3 1]
 [4 1 3 3 1 1 1 1 1 1]
 [1 4 3 3 1 1 1 1 1 1]]
查询关键词为: ['意大利封闭全国', '戈贝尔确诊新冠', '欧洲杯推迟', '汤姆汉克斯确诊', '特鲁多自我隔离', '疫情防控思政大课', '美国从阿富汗撤军', '英国央行紧急降息', '苹果折叠手机专利', '魔兽世界怀旧服']
查询结果top-10: [[ 13 392 203 147  18 401 272  12 132  52]
 [272 203 100 245  91 300 111  13  64 116]
 [100 147  64  90  57  55 132 116 239  86]
 [272 100 203 245 111  90  64 116  57  56]
 [100 272 203 245  90  64 116 111  57

有652个句子（文档），那n=652 会发生什么呢？那就是把所有的都检索出来了


In [2]:
!python main.py --n 652

TOP10单词的TF-TDF如下所示
 [('阿富汗', 0.006327252874692564), ('欧洲杯', 0.005215631936116387), ('折叠', 0.0050477958652063015), ('意大利', 0.0050270258549414346), ('央行', 0.004862996707500968), ('任务', 0.004792787625878285), ('英国', 0.004705536630277911), ('戈贝尔', 0.004662253844871659), ('特鲁多', 0.004567633613773077), ('塔利班', 0.004254171083783253)]
U: (652, 652)
s: (652,)
VT: (23782, 23782)
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.536 seconds.
Prefix dict has been built successfully.
[[0 5 3 ... 9 9 9]
 [4 3 1 ... 7 7 7]
 [1 2 1 ... 7 6 7]
 [4 1 3 ... 7 7 7]
 [1 4 3 ... 7 7 7]]
查询关键词为: ['意大利封闭全国', '戈贝尔确诊新冠', '欧洲杯推迟', '汤姆汉克斯确诊', '特鲁多自我隔离', '疫情防控思政大课', '美国从阿富汗撤军', '英国央行紧急降息', '苹果折叠手机专利', '魔兽世界怀旧服']
查询结果top-652: [[ 13 392 203 ... 581 575 650]
 [272 203 100 ... 482 478 486]
 [100 147  64 ... 486 414 478]
 ...
 [486 482 485 ...  86  64 272]
 [520 518 539 ... 501  56 203]
 [575 650 637 ... 401 132 147]]
查询结果top-652准确率: [0.08128834355828221

### 查询top-n文档准确率与文档分类准确率评估方案的简单对比

在本次作业中，我们使用了两种评估方案来衡量模型的性能：查询top-n文档准确率和文档分类准确率。



**查询top-n文档准确率**是指对于每个查询关键词，我们从所有文档中选出最相关的n个文档，并计算这些文档中有多少个是与查询关键词相关的。具体来说，我们使用`search_topn_for_each_key`函数来实现这一评估方案。

**文档分类准确率**是指将每个文档分配到最相关的查询关键词类别，并计算分类的准确率。具体来说，我们使用`classification`函数来实现这一评估方案。

具体来说助教的代码里面有写 label。


这两种评估方案肯定是各有优缺点的，适用于不同的应用场景。
前者的优点大概是说可以直观地反映出模型在检索任务中的性能，特别是当用户只关心前n个结果时。然而，这种评估方案的缺点是它只考虑了前n个结果，忽略了其他可能相关的文档。

后者的优点是可以全面地衡量模型在分类任务中的性能，因为它考虑了所有文档和所有查询关键词。然而，这种评估方案的缺点是它可能会受到类别不平衡的影响，即某些类别的文档数量较少，可能会导致分类结果不准确。

